In [1]:
import time
import numpy as np
import qiskit
from qiskit.opflow import X,Z,I
from qiskit.opflow.state_fns import StateFn, CircuitStateFn

from modules.utils import *

import matplotlib.pyplot as plt

from scipy import sparse
import scipy.sparse.linalg.eigen.arpack as arp
%matplotlib inline

I am basically following this https://qiskit.org/documentation/tutorials/operators/01_operator_flow.html#Part-IV:-Converters

In [2]:
L=5
anti = -1
gx = 1e-2
gz = 1e-2

In [3]:
H = QHIsing(L,anti,np.float32(gx),np.float32(gz))

In [4]:
evo_time = qiskit.circuit.Parameter('t') # note that we will have to bind it later

In [5]:
evolution_op = (evo_time * 1j * H).exp_i()
# expH = suzi.convert(H)

In [6]:
H.__class__, evolution_op.__class__

(qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp,
 qiskit.opflow.evolutions.evolved_op.EvolvedOp)

As far as I can tell, we need to work with these StateFns instead of circuits

In [22]:
# this is a dummy circuit for the sake of checking how I can combine the evolved state with a circuit later
circ = qiskit.circuit.library.TwoLocal(5,"rx","cz", reps=1,entanglement="linear")

In [8]:
circ.draw()

┌──────────┐   ┌──────────┐                                    
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├────────────────────────────────────
     ├──────────┤ │ └──────────┘┌──────────┐                        
q_1: ┤ RX(θ[1]) ├─■──────■──────┤ RX(θ[6]) ├────────────────────────
     ├──────────┤        │      └──────────┘┌──────────┐            
q_2: ┤ RX(θ[2]) ├────────■───────────■──────┤ RX(θ[7]) ├────────────
     ├──────────┤                    │      └──────────┘┌──────────┐
q_3: ┤ RX(θ[3]) ├────────────────────■───────────■──────┤ RX(θ[8]) ├
     ├──────────┤                                │      ├──────────┤
q_4: ┤ RX(θ[4]) ├────────────────────────────────■──────┤ RX(θ[9]) ├
     └──────────┘                                       └──────────┘

In [9]:
circ_fn = CircuitStateFn(circ)

In [10]:
print(circ_fn)

CircuitStateFn(
     ┌──────────┐   ┌──────────┐                                    
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├────────────────────────────────────
     ├──────────┤ │ └──────────┘┌──────────┐                        
q_1: ┤ RX(θ[1]) ├─■──────■──────┤ RX(θ[6]) ├────────────────────────
     ├──────────┤        │      └──────────┘┌──────────┐            
q_2: ┤ RX(θ[2]) ├────────■───────────■──────┤ RX(θ[7]) ├────────────
     ├──────────┤                    │      └──────────┘┌──────────┐
q_3: ┤ RX(θ[3]) ├────────────────────■───────────■──────┤ RX(θ[8]) ├
     ├──────────┤                                │      ├──────────┤
q_4: ┤ RX(θ[4]) ├────────────────────────────────■──────┤ RX(θ[9]) ├
     └──────────┘                                       └──────────┘
)


In [11]:
evolved_processed = evolution_op @ circ_fn # first the imaginary time evolution, then the dummy processing
print(evolved_processed )

ComposedOp([
  e^(-i*1.0*I*t * (
    1.0 * ZZIII
    - 0.009999999776482582 * XIIII
    - 0.010099999776482581 * ZIIII
    + 1.0 * IZZII
    - 0.009999999776482582 * IXIII
    - 0.009999999776482582 * IZIII
    + 1.0 * IIZZI
    - 0.009999999776482582 * IIXII
    - 0.009999999776482582 * IIZII
    + 1.0 * IIIZZ
    - 0.009999999776482582 * IIIXI
    - 0.009999999776482582 * IIIZI
    - 0.009999999776482582 * IIIIX
    - 0.009999999776482582 * IIIIZ
  )),
  CircuitStateFn(
       ┌──────────┐   ┌──────────┐                                    
  q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├────────────────────────────────────
       ├──────────┤ │ └──────────┘┌──────────┐                        
  q_1: ┤ RX(θ[1]) ├─■──────■──────┤ RX(θ[6]) ├────────────────────────
       ├──────────┤        │      └──────────┘┌──────────┐            
  q_2: ┤ RX(θ[2]) ├────────■───────────■──────┤ RX(θ[7]) ├────────────
       ├──────────┤                    │      └──────────┘┌──────────┐
  q_3: ┤ RX(θ[3]) ├───────

In [12]:
trotterized_op = qiskit.opflow.evolutions.PauliTrotterEvolution(trotter_mode=qiskit.opflow.evolutions.Suzuki(order=2, reps=1)).convert(evolved_processed)
print(trotterized_op)

CircuitStateFn(
global phase: 0
     ┌──────────┐   ┌──────────┐┌──────────────────────────────┐»
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├┤ RZ(-0.00999999977648258*I*t) ├»
     ├──────────┤ │ └──────────┘└─────────┬──────────┬─────────┘»
q_1: ┤ RX(θ[1]) ├─■──────■────────────────┤ RX(θ[6]) ├──────────»
     ├──────────┤        │                └──────────┘          »
q_2: ┤ RX(θ[2]) ├────────■─────────────────────■────────────────»
     ├──────────┤                              │                »
q_3: ┤ RX(θ[3]) ├──────────────────────────────■────────────────»
     ├──────────┤                                               »
q_4: ┤ RX(θ[4]) ├───────────────────────────────────────────────»
     └──────────┘                                               »
«     ┌──────────────────────────────┐                                »
«q_0: ┤ RX(-0.00999999977648258*I*t) ├────────────────────────────────»
«     ├──────────────────────────────┤┌──────────────────────────────┐»
«q_1: ┤ RZ(-0.009999999776

In [13]:
bound = trotterized_op.bind_parameters({evo_time: .5j})

In [21]:
bound.to_circuit().draw()

┌──────────┐   ┌──────────┐┌─────────────────────────┐»
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├┤ RZ(0.00499999988824129) ├»
     ├──────────┤ │ └──────────┘└───────┬──────────┬──────┘»
q_1: ┤ RX(θ[1]) ├─■──────■──────────────┤ RX(θ[6]) ├───────»
     ├──────────┤        │              └──────────┘       »
q_2: ┤ RX(θ[2]) ├────────■───────────────────■─────────────»
     ├──────────┤                            │             »
q_3: ┤ RX(θ[3]) ├────────────────────────────■─────────────»
     ├──────────┤                                          »
q_4: ┤ RX(θ[4]) ├──────────────────────────────────────────»
     └──────────┘                                          »
«     ┌─────────────────────────┐                           »
«q_0: ┤ RX(0.00499999988824129) ├───────────────────────────»
«     ├─────────────────────────┤┌─────────────────────────┐»
«q_1: ┤ RZ(0.00499999988824129) ├┤ RX(0.00499999988824129) ├»
«     └───────┬──────────┬──────┘├─────────────────────────┤»
«q_2: ────────┤ RX(θ[7]) ├───────┤ RZ(0.00499999988824129) ├»
«             └──────────┘       └───────┬──────────┬──────┘»
«q_3: ─────────────■─────────────────────┤ RX(θ[8]) ├───────»
«                  │                     ├──────────┤       »
«q_4: ─────────────■─────────────────────┤ RX(θ[9]) ├───────»
«                                        └──────────┘       »
«                ┌───┐            ┌────────────────────────┐┌───┐     »
«q_0: ───────────┤ X ├────────────┤ RZ(-0.500000000000000) ├┤ X ├─────»
«                └─┬─┘            └────────────────────────┘└─┬─┘┌───┐»
«q_1: ─────────────■──────────────────────────────────────────■──┤ X ├»
«     ┌─────────────────────────┐                                └─┬─┘»
«q_2: ┤ RX(0.00499999988824129) ├──────────────────────────────────■──»
«     ├─────────────────────────┤┌─────────────────────────┐          »
«q_3: ┤ RZ(0.00499999988824129) ├┤ RX(0.00499999988824129) ├──────────»
«     ├─────────────────────────┤├─────────────────────────┤          »
«q_4: ┤ RZ(0.00504999988824129) ├┤ RX(0.00499999988824129) ├──────────»
«     └─────────────────────────┘└─────────────────────────┘          »
«                                                                             »
«q_0: ────────────────────────────────────────────────────────────────────────»
«     ┌────────────────────────┐┌───┐                                         »
«q_1: ┤ RZ(-0.500000000000000) ├┤ X ├─────────────────────────────────────────»
«     └────────────────────────┘└─┬─┘┌───┐┌────────────────────────┐┌───┐     »
«q_2: ────────────────────────────■──┤ X ├┤ RZ(-0.500000000000000) ├┤ X ├─────»
«                                    └─┬─┘└────────────────────────┘└─┬─┘┌───┐»
«q_3: ─────────────────────────────────■──────────────────────────────■──┤ X ├»
«                                                                        └─┬─┘»
«q_4: ─────────────────────────────────────────────────────────────────────■──»
«                                                                             »
«                                                                        »
«q_0: ───────────────────────────────────────────────────────────────────»
«                                                                        »
«q_1: ───────────────────────────────────────────────────────────────────»
«                                                                        »
«q_2: ───────────────────────────────────────────────────────────────────»
«     ┌────────────────────────┐┌───┐┌───┐┌────────────────────────┐┌───┐»
«q_3: ┤ RZ(-0.500000000000000) ├┤ X ├┤ X ├┤ RZ(-0.500000000000000) ├┤ X ├»
«     └────────────────────────┘└─┬─┘└─┬─┘└────────────────────────┘└─┬─┘»
«q_4: ────────────────────────────■────■──────────────────────────────■──»
«                                                                        »
«                                                                »
«q_0: ───────────────────────────────────────────────────────────»
«                    